In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot
import os
from copy import deepcopy

from time import time

from math import ceil
from scipy.stats import spearmanr, gamma, poisson

from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch import tensor
from torch.cuda import is_available

from sciPENN.sciPENN_API import sciPENN_API

In [2]:
"""Read in Raw Data"""

adata_gene = sc.read("/home/asmauger/biostat625final/pbmc_gene.h5ad")
adata_protein = sc.read("/home/asmauger/biostat625final/pbmc_protein.h5ad")

## Scenario 1: Training CITE-seq, Test scRNA-seq

For this scenario, we designate patients P1, P3, P4, P7 as the training data. The other patients are treated as an scRNA-seq test set (proteins are held out).

In [3]:
"""Create training and test"""

# define training set to be donors p1, p3
# this is deviating from the tutorial to make the size more manageable
train_bool = [x in ['P1', 'P3'] for x in adata_gene.obs['donor']]
# filter gene data down to these donors
adata_gene_train = adata_gene[train_bool].copy() 
# filter protein data down to these donors
adata_protein_train = adata_protein[train_bool].copy()

# filter gene data down to donors p5 and p7 for testing
# again, this is deviating from tutorial to make size more manageable
test_bool = [x in ['P5', 'P7'] for x in adata_gene.obs['donor']]
adata_gene_test = adata_gene[train_bool].copy()
# there is no protein test set (since we are trying to predict proteins)

adata_gene_train = adata_gene_train[:, 10000:11000]
adata_gene_test = adata_gene_test[:, 10000:11000]


Create the sciPENN object. Since we only have a single CITE-seq reference, we provide only a single gene training set (represented by the list of length 1 passed to gene_trainsets) and a single protein training set (represented by the list of length 1 passed to protein_trainsets). 

The key used to identify batches is 'donor' for both the CITE-seq reference and the scRNA-seq query, so we pass this key to both the train_batchkeys argument and the test_batchkey argument.

Lastly, we also want to transfer celltype labels from CITE-seq reference to query. The celltypes in the CITE-seq reference are identified by the obs key "celltype.l3", so we supply this string to the type_key argument.

In [4]:
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_train], protein_trainsets = [adata_protein_train], 
                      gene_test = adata_gene_test, train_batchkeys = ['donor'], test_batchkey = 'donor',  use_gpu=False,
                     select_hvg=False)

Using CPU


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.



QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells

Log-Normalizing Training Data
Log-Normalizing Testing Data

Normalizing Gene Training Data by Batch


100%|██████████| 2/2 [00:00<00:00, 47.71it/s]



Normalizing Protein Training Data by Batch


100%|██████████| 2/2 [00:00<00:00, 49.04it/s]



Normalizing Gene Testing Data by Batch


100%|██████████| 2/2 [00:00<00:00, 49.45it/s]


View of AnnData object with n_obs × n_vars = 32795 × 1000
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'batch'
    var: 'name'
    obsm: 'X_apca', 'X_aumap', 'X_pca', 'X_spca', 'X_umap', 'X_wnn.umap'

Train the sciPENN object. Here, we ask the model to estimate quantiles 0.1, 0.25, 0.75, and 0.9. The weights directory is "pbmc_to_pbmc"

In [5]:
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), weights_dir = "pbmc_to_pbmc", load = True)

Epoch 0 prediction loss = 1.465
Epoch 1 prediction loss = 1.456
Epoch 2 prediction loss = 1.440
Epoch 3 prediction loss = 1.407
Epoch 4 prediction loss = 1.350
Epoch 5 prediction loss = 1.279
Epoch 6 prediction loss = 1.219
Epoch 7 prediction loss = 1.183
Epoch 8 prediction loss = 1.167
Epoch 9 prediction loss = 1.164
Epoch 10 prediction loss = 1.172
Epoch 11 prediction loss = 1.180
Epoch 12 prediction loss = 1.182
Epoch 13 prediction loss = 1.176
Decaying loss to 0.0001
Epoch 14 prediction loss = 1.175
Epoch 15 prediction loss = 1.172
Epoch 16 prediction loss = 1.171
Epoch 17 prediction loss = 1.169
Epoch 18 prediction loss = 1.166
Epoch 19 prediction loss = 1.164
Decaying loss to 1e-05
Epoch 20 prediction loss = 1.164


Use the predict method to predict protein expression and celltype labels in the scRNA-seq test dataset.

In [6]:
predicted_test = sciPENN.predict()

The predicted protein data is accessible via attribute X.

In [7]:
predicted_test.X

array([[-0.7746329 , -0.65737534, -0.13420999, ...,  0.07423229,
        -0.4964252 ,  0.12834555],
       [-0.4914093 ,  0.37110937, -0.63508147, ..., -0.5585097 ,
         0.44774455, -0.2483236 ],
       [-0.36307186, -0.24987161, -0.68349564, ..., -0.70489234,
         1.5201046 , -1.0842584 ],
       ...,
       [-0.04042007,  0.83019274,  0.03713248, ..., -0.30181816,
         0.15947166, -0.25917786],
       [-0.17794195,  0.5090235 ,  0.42953056, ..., -0.33902368,
         0.12975785, -0.41442466],
       [ 0.02950515,  0.6669995 ,  0.01941851, ..., -0.06443219,
         0.37494168,  0.11847007]], dtype=float32)

Use the 'batch' cell metadata field to see which batch a cell is from. E.g. the batch information 'DS-Test P5' means the corresponding cell is from batch P5 of the query dataset.

In [8]:
predicted_test.obs['batch']

index
L1_AATGACCAGTTGAAAC    DS-Test P1
L1_ACTCCCACAAGCAGGT    DS-Test P3
L1_AGACACTTCGGCTGAC    DS-Test P1
L1_AGGGAGTAGTACGAGC    DS-Test P1
L1_AGGGTCCAGACTACCT    DS-Test P3
                          ...    
L5_TACCCACAGCAACCAG    DS-Test P3
L5_TCTCACGCACTTCATT    DS-Test P1
L5_TGATTTCCATATAGCC    DS-Test P1
L5_TTCAATCCACAGAAGC    DS-Test P1
L5_TTCGGTCCAGGCACAA    DS-Test P3
Name: batch, Length: 113, dtype: object

Use the 'transfered cell labels' cell metadata field to see the predicted celltype for each cell. We actually know the true celltype label in this case (found in celltype.l3) so we can view the test accuracy of predictions.

In [9]:
print(predicted_test.obs['transfered cell labels'])
print(f"\n\n\nTest set accuracy: {(predicted_test.obs['transfered cell labels'] == predicted_test.obs['celltype.l3']).mean():.4f}")

KeyError: 'transfered cell labels'

Use the embed command to estimate a lower-dimension latent representation of the data. Both the CITE-seq reference and scRNA-seq query are embedded into a common latent space, which can be further dimension reduced and visualized using UMAP.

In [15]:
embedding = sciPENN.embed()

The embedding is accessible via attribute X.

In [16]:
predicted_test.X

array([[-0.7746329 , -0.65737534, -0.13420999, ...,  0.07423229,
        -0.4964252 ,  0.12834555],
       [-0.4914093 ,  0.37110937, -0.63508147, ..., -0.5585097 ,
         0.44774455, -0.2483236 ],
       [-0.36307186, -0.24987161, -0.68349564, ..., -0.70489234,
         1.5201046 , -1.0842584 ],
       ...,
       [-0.04042007,  0.83019274,  0.03713248, ..., -0.30181816,
         0.15947166, -0.25917786],
       [-0.17794195,  0.5090235 ,  0.42953056, ..., -0.33902368,
         0.12975785, -0.41442466],
       [ 0.02950515,  0.6669995 ,  0.01941851, ..., -0.06443219,
         0.37494168,  0.11847007]], dtype=float32)

Use the 'batch' cell metadata field to see which dataset and batch a cell is from. E.g. the batch information 'DS-Test P5' means the corresponding cell is from batch P5 of the query dataset. 'DS-1 P4' means the corresponding cell is from batch P4 of the first CITE-seq reference.

In [17]:
embedding.obs['batch']

index
L1_AATGACCAGTTGAAAC       DS-1 P1
L1_ACTCCCACAAGCAGGT       DS-1 P3
L1_AGACACTTCGGCTGAC       DS-1 P1
L1_AGGGAGTAGTACGAGC       DS-1 P1
L1_AGGGTCCAGACTACCT       DS-1 P3
                          ...    
L5_TACCCACAGCAACCAG    DS-Test P3
L5_TCTCACGCACTTCATT    DS-Test P1
L5_TGATTTCCATATAGCC    DS-Test P1
L5_TTCAATCCACAGAAGC    DS-Test P1
L5_TTCGGTCCAGGCACAA    DS-Test P3
Name: batch, Length: 226, dtype: object

View estimated quantiles of the data. We estimate a quantile for every protein in every cell, so each of these layers is an array whose rows index cells and columns index proteins.

In [20]:
q25 = predicted_test.layers['q25']
q75 = predicted_test.layers['q75']
q10 = predicted_test.layers['q10']
q90 = predicted_test.layers['q90']
print(q25)

[[-0.8660718  -0.56719124 -0.58896625 ... -0.16141163 -0.81078225
  -0.30788174]
 [-0.6598691   0.27645323 -0.13130261 ... -0.22857639  0.20451882
  -0.6279048 ]
 [-0.72408247  0.36490095 -0.1684413  ... -0.7728595   0.5059874
  -1.2857238 ]
 ...
 [-0.1701657   0.45233893 -0.15208584 ...  0.02045268  0.35814184
  -0.5399869 ]
 [-0.43102813  0.3083251   0.0250212  ... -0.10895663  0.13762899
  -0.9771651 ]
 [-0.37057763  0.31863755 -0.2751903  ... -0.05252589  0.17315592
  -0.54936695]]


In [21]:
embedding

AnnData object with n_obs × n_vars = 226 × 512
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'batch'

## Scenario 2: Integrate two CITE-seq datasets

For this scenario, we integrate two separate CITE-seq datasets. The first consists of patients P1, P3, P4, P7, and the other CITE-seq dataset contains other patients. We use sciPENN to handle the case where the protein panels of the two CITE-seq datasets are not identical. Some proteins are availible in only one CITE-seq dataset, other proteins are availible only in the other dataset. The goal here is to recover the missing proteins in each dataset so that the two CITE-seq datasets can be merged

In [14]:
"""Create training and test"""

train_bool = [x in ['P1', 'P3', 'P4', 'P7'] for x in adata_gene.obs['donor']]

adata_gene_set1 = adata_gene[train_bool].copy()
adata_protein_set1 = adata_protein[train_bool].copy()
adata_gene_set2 = adata_gene[np.invert(train_bool)].copy()
adata_protein_set2 = adata_protein[np.invert(train_bool)].copy()

common_proteins = adata_protein_train.var.index
set1only_proteins = np.random.choice(common_proteins, len(common_proteins)//3, False)
common_proteins = np.setdiff1d(common_proteins, set1only_proteins)
set2only_proteins = np.random.choice(common_proteins, len(common_proteins)//2, False)

set1only_proteins = set(set1only_proteins)
set2only_proteins = set(set2only_proteins)

keep_set1 = [x not in set2only_proteins for x in adata_protein_train.var.index]
keep_set2 = [x not in set1only_proteins for x in adata_protein_train.var.index]

adata_protein_set1 = adata_protein_set1[:, keep_set1].copy()
adata_protein_set2 = adata_protein_set2[:, keep_set2].copy()

Create the sciPENN object. Since we two CITE-seq references and no scRNA-seq query set this time, we provide two gene training sets (one for each CITE-seq reference) and a two protein training sets (the corrresponding protein arrays of the two CITE-seq references). Since we have no scRNA-seq query set, do not provide a gene_test argument.

Since we have two CITE-seq references, we need to provide a list of two batch keys for the train_batchkeys argument, which lists the batchkeys for each of the two references. Since the batchkey is 'donor' for both datasets, the two entries in the list are both "donor". Since we have no query scRNA-seq dataset, do not provide a test_batchkey argument.

For this scenario, we do not want to transfer celltype labels from CITE-seq reference to query. The celltypes is not provided.

In [15]:
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_set1, adata_gene_set2], 
                      protein_trainsets = [adata_protein_set1, adata_protein_set2], 
                      train_batchkeys = ['donor', 'donor'])

Searching for GPU
GPU not detected, falling back to CPU

QC Filtering Training Cells

QC Filtering Training Genes

Normalizing Training Cells

Log-Normalizing Training Data

Finding HVGs


... storing 'orig.ident' as categorical
... storing 'donor' as categorical
... storing 'batch' as categorical
... storing 'Dataset' as categorical
/Users/jlakkis/anaconda3/envs/scipennenv/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



Normalizing Gene Training Data by Batch


100%|█████████████████████████████████████████████| 8/8 [00:02<00:00,  3.78it/s]



Normalizing Protein Training Data by Batch


100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  3.71it/s]


Train the sciPENN object. Here, we ask the model to estimate quantiles 0.1, 0.25, 0.75, and 0.9. The weights directory is "pbmc_to_pbmcINTEGRATE"

In [16]:
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), weights_dir = "pbmc_to_pbmcINTEGRATE", load = True)

Use the impute method to impute missing protein expression in each CITE-seq reference set. Note that this is different from the predict method, which would predict protein expression only for the gene test set and return an array with the same number of rows as the test set. The impute function returns an array of n x p, where n is the sum of the number of cells across all CITE-seq references and p is the union of the sets of proteins across al CITE-seq references. A protein will be imputed for a cell in this array only if it wasn't sequenced for that cell. Otherwise the true sequenced value is provided.

In [17]:
imputed_test = sciPENN.impute()

The integrated protein reference data is accessible via attribute X.

In [18]:
imputed_test.X

array([[ 4.48758692e-01, -8.01393747e-01, -3.42670083e-01, ...,
        -4.10658121e-01, -5.39497077e-01, -6.77482486e-02],
       [-4.67995927e-02,  1.12219346e+00, -4.16118801e-02, ...,
        -3.67593259e-01,  1.34710538e+00,  2.94366851e-03],
       [ 8.93130660e-01,  2.32020020e-01,  2.72543192e-01, ...,
        -4.26250935e-01,  8.23240519e-01,  1.39810681e-01],
       ...,
       [ 1.02716215e-01,  1.30520761e-03,  7.59903312e-01, ...,
        -2.01725900e-01, -1.95526272e-01,  1.23690259e+00],
       [-3.45076263e-01, -2.60932874e-02,  9.24792290e-01, ...,
         3.67230892e-01,  3.22137713e-01,  9.67191458e-01],
       [ 1.25230563e+00, -4.54774916e-01,  1.19021249e+00, ...,
         2.84777462e-01,  1.17554748e+00, -1.42882025e+00]], dtype=float32)

We can use the 'Dataset' cell metadata field to see which CITE-seq dataset each cell is from. Dataset 1 refers to the first CITE-seq dataset (whose gene data was the first element in the gene_trainsets list and whose protein data was the first element in the protein_trainsets list).

In [19]:
imputed_test.obs['Dataset']

index
L1_AAACCCAAGACATACA-0      Dataset 1
L1_AAACCCACAACTGGTT-0      Dataset 1
L1_AAACCCACACGTACTA-0      Dataset 1
L1_AAACCCACAGCATACT-0      Dataset 1
L1_AAACCCACATCAGTCA-0      Dataset 1
                             ...    
E2L8_TTTGTTGGTCGTGATT-1    Dataset 2
E2L8_TTTGTTGGTGTGCCTG-1    Dataset 2
E2L8_TTTGTTGGTTAGTTCG-1    Dataset 2
E2L8_TTTGTTGGTTGGCTAT-1    Dataset 2
E2L8_TTTGTTGTCTCATGGA-1    Dataset 2
Name: Dataset, Length: 161748, dtype: object

Each dataset has batches inside it. Use the 'batch' cell metadata field to see which batch a cell is from. E.g. the batch information 'DS-1 P4' means the corresponding cell is from batch P4 of dataset 1.

In [20]:
imputed_test.obs['batch']

index
L1_AAACCCAAGACATACA-0      DS-1 P1
L1_AAACCCACAACTGGTT-0      DS-1 P4
L1_AAACCCACACGTACTA-0      DS-1 P3
L1_AAACCCACAGCATACT-0      DS-1 P4
L1_AAACCCACATCAGTCA-0      DS-1 P3
                            ...   
E2L8_TTTGTTGGTCGTGATT-1    DS-2 P5
E2L8_TTTGTTGGTGTGCCTG-1    DS-2 P5
E2L8_TTTGTTGGTTAGTTCG-1    DS-2 P8
E2L8_TTTGTTGGTTGGCTAT-1    DS-2 P5
E2L8_TTTGTTGTCTCATGGA-1    DS-2 P5
Name: batch, Length: 161748, dtype: object

We may want a quick way to identify which proteins were sequenced for a particular dataset. The following commands demontrate how to do this.

In [21]:
proteins = imputed_test.var.index

proteins1 = proteins[imputed_test.var['Dataset 1']] #get proteins sequenced in Dataset 1
proteins2 = proteins[imputed_test.var['Dataset 2']] #get proteins sequenced in Dataset 1

Recall that the overall array (imputed_test.X) consists of both imputed and true protein values, with imputed values being used when the protein wasn't sequenced. We can separate this array into 4 subarrays:

1. Imputed Protein array in Dataset 1
2. Sequenced Protein array in Dataset 1
3. Imputed Protein array in Dataset 2
4. Sequenced Protein array in Dataset 2

In [22]:
ds1_cells = imputed_test.obs['Dataset'] == 'Dataset 1'
ds2_cells = imputed_test.obs['Dataset'] == 'Dataset 2'

ds1_pred, ds1_seq = np.invert(imputed_test.var['Dataset 1']), imputed_test.var['Dataset 1']
ds2_pred, ds2_seq = np.invert(imputed_test.var['Dataset 2']), imputed_test.var['Dataset 2']

pred1 = imputed_test[ds1_cells, ds1_pred] #imputed protein array in dataset 1
sequenced1 = imputed_test[ds1_cells, ds1_seq] #sequenced protein array in dataset 1
pred2 = imputed_test[ds2_cells, ds2_pred] #imputed protein array in dataset 2
sequenced2 = imputed_test[ds2_cells, ds2_seq] #sequenced protein array in dataset 2

We can embed the two references into a common latent space. The embedding process is identical to scenario 1.

In [23]:
embedding = sciPENN.embed()

Use the 'batch' cell metadata field to see which dataset and batch a cell is from. E.g.'DS-1 P4' means the corresponding cell is from batch P4 of the first CITE-seq reference. 'DS-2 P5' means the corresponding cell is from batch P5 of the first CITE-seq reference. 

In [24]:
embedding.obs['batch']

index
L1_AAACCCAAGACATACA-0      DS-1 P1
L1_AAACCCACAACTGGTT-0      DS-1 P4
L1_AAACCCACACGTACTA-0      DS-1 P3
L1_AAACCCACAGCATACT-0      DS-1 P4
L1_AAACCCACATCAGTCA-0      DS-1 P3
                            ...   
E2L8_TTTGTTGGTCGTGATT-1    DS-2 P5
E2L8_TTTGTTGGTGTGCCTG-1    DS-2 P5
E2L8_TTTGTTGGTTAGTTCG-1    DS-2 P8
E2L8_TTTGTTGGTTGGCTAT-1    DS-2 P5
E2L8_TTTGTTGTCTCATGGA-1    DS-2 P5
Name: batch, Length: 161748, dtype: object

The 'Dataset' attribute is a quick way to check a cell's source dataset.

In [25]:
embedding.obs['Dataset']

index
L1_AAACCCAAGACATACA-0      Dataset 1
L1_AAACCCACAACTGGTT-0      Dataset 1
L1_AAACCCACACGTACTA-0      Dataset 1
L1_AAACCCACAGCATACT-0      Dataset 1
L1_AAACCCACATCAGTCA-0      Dataset 1
                             ...    
E2L8_TTTGTTGGTCGTGATT-1    Dataset 2
E2L8_TTTGTTGGTGTGCCTG-1    Dataset 2
E2L8_TTTGTTGGTTAGTTCG-1    Dataset 2
E2L8_TTTGTTGGTTGGCTAT-1    Dataset 2
E2L8_TTTGTTGTCTCATGGA-1    Dataset 2
Name: Dataset, Length: 161748, dtype: object

We can view estimated quantiles just like in Scenario 1. Note that each quantile is estimated for all proteins in. all cells, even when the protein is sequenced in the cell. The quantiles are much more useful for proteins which needed to be imputed for a cell.

In [26]:
q10_pred = imputed_test[ds1_cells, ds1_pred].layers['q10'] #get q10 for imputed proteins from reference 1
q10_truth = imputed_test[ds1_cells, ds1_seq].layers['q10'] #get q10 for sequenced proteins from reference 1, not useful